In [9]:
!python3.11 -m pip install faker numpy pandas

  Using cached Faker-18.7.0-py3-none-any.whl (1.7 MB)
  Using cached pandas-2.0.1-cp311-cp311-win_amd64.whl (10.6 MB)


ERROR: Could not install packages due to an OSError: [WinError 2] Le fichier spécifié est introuvable: 'c:\\python311\\Scripts\\faker.exe' -> 'c:\\python311\\Scripts\\faker.exe.deleteme'


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from faker import Faker
import numpy as np
import pandas as pd
import random
import datetime
fake = Faker("fr-FR")

In [2]:
def list_to_sql(liste,table):
    strings = []
    for e in liste:
        strings.append(f"INSERT INTO {table} ({', '.join(e.keys())}) VALUES {str(tuple(e.values()))};")
    return "\n".join(strings)

In [11]:
clients = []
for i in range(0,1000):
    client = {"id_client":fake.pystr(),"prenom":fake.first_name(),"nom":fake.last_name(),"mdp":fake.password(),"tel":fake.phone_number(),"adresse_factu":fake.address(),"num_carte":fake.credit_card_number(),"statut":"BASIC"}
    client["mail"] = client["prenom"]+"."+client["nom"]+"@gmail.com"
    clients.append(client)

produits = [
    {"id_produit":0,"nom":"Gerbera","prix":5,"disponibilite":"A l'année"},
    {"id_produit":1,"nom":"Ginger","prix":4,"disponibilite":"A l'année"},
    {"id_produit":2,"nom":"Glaïeul","prix":1,"disponibilite":"Mai à novembre"},
    {"id_produit":3,"nom":"Marguerite","prix":2.25,"disponibilite":"A l'année"},
    {"id_produit":4,"nom":"Rose rouge","prix":2.5,"disponibilite":"A l'année"}
]

bouquets_standards = [
    {"id_bs":0,"nom":"Gros Merci","prix":45,"composition":"Arrangement floral avec marguerites et verdure","categorie":"Toute occasion"},
    {"id_bs":1,"nom":"L'amoureux","prix":65,"composition":"Arrangement floral avec roses blanches et roses","categorie":"St-Valentin"},
    {"id_bs":2,"nom":"L'Exotique","prix":40,"composition":"Arrangement floral avec ginger, oiseaux du paradis, roses et genet","categorie":"Toute occasion"},
    {"id_bs":3,"nom":"Maman","prix":80,"composition":"Arrangement floral avec gerbera, roses blanches, lys et alstroméria","categorie":"Fête des mères"},
    {"id_bs":4,"nom":"Vive la mariée","prix":120,"composition":"Arrangement floral avec lys et orchidées","categorie":"Mariage"}
]

boutiques = []
for i in range(10):
    boutiques.append({"id_boutique":fake.pystr(),"adresse":fake.address()})

commandes = []
bouquets_persos = []
composition_bouquets = []

for i in range(0,2000):


    prix_tot=0

    if np.random.randint(0,2)==1:
        bs = np.random.choice(bouquets_standards,1)[0]
        prix_tot = bs["prix"]
        id_bouquet = bs["id_bs"]
        est_standard=True
    else: 
        produits_bouquet = np.random.choice([x for x in produits],np.random.randint(1,4), replace=False)
        bouquet_perso = {"id_bp":fake.pystr(),"description_bp":"Arrangement floral avec "+", ".join([x["nom"] for x in produits_bouquet]), "prix_max":round(np.random.randint(1,20))*10}
        bouquets_persos.append(bouquet_perso)
        id_bouquet = bouquet_perso["id_bp"]

        for produit in produits_bouquet:
            composition_bouquet = {"id_bp":id_bouquet,"id_produit":produit["id_produit"],"quantite":round((bouquet_perso["prix_max"]/len(produits_bouquet))/produit["prix"])}
            composition_bouquets.append(composition_bouquet)
            prix_tot+=produit["prix"]*composition_bouquet["quantite"]
        est_standard=False

    commande = {"id_commande":fake.pystr(),"id_client":np.random.choice(clients,1)[0]["id_client"],"id_boutique":random.choice(boutiques)["id_boutique"],"id_bouquet":id_bouquet,"est_standard":est_standard,"date_commande":fake.date_between(datetime.datetime.strptime("2022/01/01","%Y/%m/%d"))}
    
    if commande["est_standard"]:
        commande["etat"] = random.choice(["CC","VINV","CAL","CL"])
    else: 
        commande["etat"] = random.choice(["CC","CPAV","CAL","CL"])

    if commande["etat"]=="CL":
        commande["date_livraison"] = (commande["date_commande"]+datetime.timedelta(days=np.random.randint(20))).strftime("%Y/%m/%d")
    commande["date_commande"] = commande["date_commande"].strftime("%Y/%m/%d")
    commande["prix_tot"] = prix_tot
    commandes.append(commande)


stocks = []
for boutique in boutiques:
    for produit in produits:
        stocks.append({"id_boutique":boutique["id_boutique"],"id_produit":produit["id_produit"],"quantite":np.random.randint(500)})

In [12]:
sql_clients = list_to_sql(clients,"client")
sql_produits = list_to_sql(produits,"produit")
sql_bouquet_standards = list_to_sql(bouquets_standards,"bouquet_standard")
sql_bouquet_perso = list_to_sql(bouquets_persos,"bouquet_perso")
sql_commandes = list_to_sql(commandes,"commande").replace(", None",", null")
sql_composition_bouquets = list_to_sql(composition_bouquets,"composition_bouquet")
sql_boutiques = list_to_sql(boutiques,"boutique")
sql_stocks = list_to_sql(stocks,"stock")

In [13]:
sql = "use fleur;\n\n"+"\n\n".join([sql_clients,sql_produits,sql_bouquet_standards,sql_bouquet_perso,sql_boutiques,sql_commandes,sql_composition_bouquets,sql_stocks])
with open("data.sql","w+") as f:  
    f.write(sql)